In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/massil/repos/newsvendor_tester


In [2]:
import scipy.stats
import numpy as np

from algorithms.cup import CUP_algorithm
from environment import *

In [3]:
nb_products = 1
horizon = 100_000
holding_costs = np.ones(nb_products)
penalty_costs = 2*np.ones(nb_products)

np.random.seed(1)
demands = np.zeros((horizon+1,nb_products))
demands[1:,:] = scipy.stats.randint(low=1,high=4).rvs(size=(horizon, nb_products))
env = Environment_Perishable_Newsvendor(3,demands,holding_costs,penalty_costs)

In [4]:
yt = np.zeros(nb_products)
for t in range(1,11) :
    xt = env.get_state(t,yt)
    demand = env.demands[t]
    yt = 3*np.ones(1)
    sales = env.get_sales(t,yt)
    print("t={}, true state={},\n\txt={}, yt={}, demand={}, sales={}".format(t,env.state, xt,yt,demand,sales))

t=1, true state=[[0. 0.]],
	xt=[0.], yt=[3.], demand=[2.], sales=[2.]
t=2, true state=[[0. 1.]],
	xt=[1.], yt=[3.], demand=[1.], sales=[1.]
t=3, true state=[[0. 2.]],
	xt=[2.], yt=[3.], demand=[1.], sales=[1.]
t=4, true state=[[1. 2.]],
	xt=[2.], yt=[3.], demand=[2.], sales=[2.]
t=5, true state=[[0. 1.]],
	xt=[1.], yt=[3.], demand=[2.], sales=[2.]
t=6, true state=[[0. 1.]],
	xt=[1.], yt=[3.], demand=[1.], sales=[1.]
t=7, true state=[[0. 2.]],
	xt=[2.], yt=[3.], demand=[1.], sales=[1.]
t=8, true state=[[1. 2.]],
	xt=[2.], yt=[3.], demand=[2.], sales=[2.]
t=9, true state=[[0. 1.]],
	xt=[1.], yt=[3.], demand=[1.], sales=[1.]
t=10, true state=[[0. 2.]],
	xt=[2.], yt=[3.], demand=[2.], sales=[2.]


### Simulation of CUP

In [7]:
env.reset()

initial_decision = 0*np.ones(nb_products)
y_min = 0*np.ones(nb_products)
y_max = 3*np.ones(nb_products)
alg = CUP_algorithm(initial_decision,y_min,y_max,1/np.sqrt((y_max-y_min)*(y_max-y_min)), np.sqrt(nb_products)*np.max(np.maximum(holding_costs,penalty_costs)))

yt, gt, st, dt = np.zeros(nb_products), np.zeros(nb_products), np.zeros(nb_products), np.zeros(nb_products)
for t in range(1,11) :
    print("t={}".format(t))

    xt = env.get_state(t,yt)
    if((xt==0).all()) :
        print("\tEmpty system")

    new_yt = alg.next_decision(t,xt,gt,st,dt)
    if((yt!=new_yt).any()) :
        print("\tChanging yt from {} to {}".format(yt,new_yt))
        yt = new_yt
    
    lt = env.get_loss(t,yt)
    gt = env.get_subgradient(t,yt)
    st = env.get_sales(t,yt)
    dt = env.get_demand(t)

t=1
	Empty system
t=2
	Empty system
	Changing yt from [0.] to [1.]
t=3
	Empty system
	Changing yt from [1.] to [1.70710678]
t=4
t=5
	Empty system
	Changing yt from [1.70710678] to [1.99578192]
t=6
	Empty system
	Changing yt from [1.99578192] to [2.49578192]
t=7
t=8
t=9
t=10
